In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,50221
2,Huelva,Confirmados PDIA 14 días,2408
3,Huelva,Tasa PDIA 14 días,"469,2402127949802"
4,Huelva,Confirmados PDIA 7 días,1502
5,Huelva,Tasa PDIA 7 dias,"292,69053140284893"
6,Huelva,Total Confirmados,50432
7,Huelva,Curados,46656
8,Huelva,Fallecidos,425


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  50221.0


/tmp/ipykernel_3312/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  14066.0


/tmp/ipykernel_3312/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3312/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3312/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3312/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 289 personas en los últimos 7 días 

Un positivo PCR cada 183 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,50221.0,1502.0,2408.0,513170.0,292.690531,469.240213,201.0
Huelva-Costa,29602.0,851.0,1346.0,289548.0,293.906364,464.862475,115.0
Huelva (capital),14066.0,498.0,786.0,143837.0,346.225241,546.451887,73.0
Condado-Campiña,15561.0,378.0,641.0,156231.0,241.949421,410.289891,50.0
Sierra de Huelva-Andévalo Central,4628.0,272.0,419.0,67391.0,403.614726,621.744743,44.0
Aracena,850.0,119.0,169.0,8255.0,1441.550575,2047.244094,18.0
Aljaraque,1828.0,69.0,121.0,21474.0,321.318804,563.472106,16.0
Palos de la Frontera,1249.0,31.0,67.0,11742.0,264.009538,570.601260,9.0
Cortegana,338.0,26.0,34.0,4602.0,564.971751,738.809213,8.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Puerto Moral,16.0,5.0,11.0,278.0,1798.561151,3956.834532,1.0
Cumbres Mayores,178.0,20.0,39.0,1749.0,1143.510577,2229.845626,0.0
Aracena,850.0,119.0,169.0,8255.0,1441.550575,2047.244094,18.0
Marines (Los),21.0,6.0,6.0,399.0,1503.759398,1503.759398,0.0
Corteconcepción,55.0,5.0,8.0,536.0,932.835821,1492.537313,4.0
Villanueva de las Cruces,25.0,3.0,4.0,387.0,775.193798,1033.591731,0.0
Fuenteheridos,21.0,3.0,6.0,709.0,423.131171,846.262341,0.0
Bollullos Par del Condado,1491.0,70.0,111.0,14387.0,486.550358,771.529853,7.0
Cortegana,338.0,26.0,34.0,4602.0,564.971751,738.809213,8.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Villablanca,288.0,1.0,3.0,2885.0,34.662045,103.986135,0.0,0.333333
Palma del Condado (La),1522.0,13.0,37.0,10801.0,120.359226,342.560874,3.0,0.351351
Valverde del Camino,946.0,20.0,56.0,12750.0,156.862745,439.215686,0.0,0.357143
Paterna del Campo,287.0,7.0,18.0,3457.0,202.487706,520.682673,0.0,0.388889
Zalamea la Real,133.0,2.0,5.0,3054.0,65.487885,163.719712,2.0,0.400000
Puerto Moral,16.0,5.0,11.0,278.0,1798.561151,3956.834532,1.0,0.454545
Palos de la Frontera,1249.0,31.0,67.0,11742.0,264.009538,570.601260,9.0,0.462687
Cerro de Andévalo (El),202.0,1.0,2.0,2327.0,42.973786,85.947572,0.0,0.500000
Santa Bárbara de Casa,49.0,1.0,2.0,1043.0,95.877277,191.754554,1.0,0.500000
